In [8]:
import specpy

from pipeline2.taskgeneration.coordinate_building_blocks import StagePositionListGenerator, StageOffsetsSettingsGenerator
from pipeline2.taskgeneration.parameter_filtering import DefaultLocationRemover
from pipeline2.callback_buildingblocks.static_settings import FOVSettingsGenerator
from pipeline2.detection import SimpleFocusPlaneDetector, SimpleManualOffset
from pipeline2.callback_buildingblocks.data_selection import NewestDataSelector
from pipeline2.utils.tiling import centered_tiles
from pipeline2.callback_buildingblocks.static_settings import JSONSettingsLoader
from pipeline2.taskgeneration import AcquisitionTaskGenerator
from pipeline2.pipeline import AcquisitionPipeline
from pipeline2.imspector import get_current_stage_coords

In [5]:
# where to save & whether to save combined HDF5 file
save_folder = 'D:/AUTOMATION/Gabi/GS487_Dppa3_all_mESC/raw'
save_hdf5 = True

# path of measurement parameters (dumped to JSON file)
measurement_parameters = 'C:/Users/RESOLFT/Desktop/config_json/gabi/20240307_590_480_overview.json'

# yx FOV size
fov_size = [50e-6, 50e-6]

# yx number of tiles
n_tiles = [20, 20]

# how much the tiles should overlap (0-1)
overlap_fraction = 0

# channel to focus in
focus_channel = 0

# manual offset (zyx) to focus
manual_focus_offset = [0, 0, 0]

In [6]:
# get current coordinates and print, so we can go back to that position
start_coords = get_current_stage_coords()
print(start_coords)

# generate regular grid around current stage position
coordinate_list = centered_tiles(start_coords, length=[0]+fov_size, n_tiles=[1]+n_tiles, overlap=overlap_fraction)

[0.00186695, -0.0032719, -0.00547788]


In [7]:
# build pipeline object (1 level: 'field')
pipeline = AcquisitionPipeline('multipoint-acquisition', save_folder, ['field'], save_combined_hdf5=save_hdf5)

# callback that will create an acquisition task with given measurement parameters
# at the next stage coordinates in the coordinate list (the next 'position')
next_position_generator = AcquisitionTaskGenerator('field',
    DefaultLocationRemover(JSONSettingsLoader(measurement_parameters, as_measurements=False)),
    FOVSettingsGenerator(lengths=[8e-6, 50e-6, 50e-6], pixel_sizes=None),
    StageOffsetsSettingsGenerator(StagePositionListGenerator(coordinate_list)),
    StageOffsetsSettingsGenerator(
        SimpleManualOffset(
            SimpleFocusPlaneDetector(
                NewestDataSelector(pipeline, level='field'),
                channel=focus_channel
        ), offset=manual_focus_offset))
    )

# attach callback so that after each position, the next one will be enqueued
pipeline.add_callback(next_position_generator, 'field')

# start with initial task from callback
pipeline.run(next_position_generator)